<a href="https://colab.research.google.com/github/nguyenvudev20/mse22.Coursera_ML_AI/blob/main/AssignmentEnd3.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Prices Competition for Kaggle Learn Users

Housing Prices Competition for Kaggle Learn Users là một bộ dữ liệu được cung cấp bởi Kaggle, một nền tảng nổi tiếng về các cuộc thi về khoa học dữ liệu và máy học. Bộ dữ liệu này bao gồm thông tin về giá nhà ở Ames, Iowa, Hoa Kỳ.

Bộ dữ liệu này bao gồm hai tập dữ liệu, một tập dữ liệu huấn luyện và một tập dữ liệu kiểm tra. Tập dữ liệu huấn luyện bao gồm 1460 mẫu và 80 biến đầu vào, bao gồm các thông tin về kích thước, vị trí, tiện nghi và trạng thái của ngôi nhà. Biến đầu ra là giá trị của ngôi nhà. Tập dữ liệu kiểm tra bao gồm 1459 mẫu và cũng bao gồm các biến đầu vào nhưng không có biến đầu ra.

Bộ dữ liệu này được sử dụng để đào tạo các mô hình dự đoán giá nhà dựa trên các đặc trưng của ngôi nhà. Nhiệm vụ của các thí sinh là tìm ra một mô hình tốt nhất để dự đoán giá trị của các ngôi nhà trong tập dữ liệu kiểm tra. Bộ dữ liệu này là một trong những bộ dữ liệu phổ biến và được sử dụng nhiều trong lĩnh vực dự đoán giá nhà và phân tích dữ liệu.


## Table of Contents
- [1 - Cài đặt dữ liệu từ Kaggle](#1)
- [2 - Lựa chọn và chuẩn hóa dữ liệu cần cho việc training](#2)
- [3 - Thực hiện trainning dựa trên model của Tensorflow Keras](#3)
- [4 - Sử dụng Hyperparameter Tuning để cải tiến model](#4)
- [5 - Cải tiến Hyperparameter Tuning](#5)
- [6 - Cải tiến bằng cách dùng Hyperband](#6)

<a name='1'></a>
## 1 - Cài đặt dữ liệu từ Kaggle

In [1]:
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import numpy as np
# tạo các tham số môi trường cho api kaggle để kết nối tải file 
os.environ['KAGGLE_USERNAME'] = "vunguyen2020"
os.environ['KAGGLE_KEY'] = "ceaa092a89951b4d01485d4ee50cf382"

In [3]:
!kaggle competitions download -c home-data-for-ml-course #tải gói data cho bài tutorial từ kaggle dưới dạng zip file
!mkdir kaggle #tạo folder để unzip file vừa tải
!unzip /content/home-data-for-ml-course.zip -d kaggle #tiến hành unzip vào folder kaggle

100% 386k/386k [00:00<00:00, 728kB/s]
100% 386k/386k [00:00<00:00, 727kB/s]
Archive:  /content/home-data-for-ml-course.zip
  inflating: kaggle/data_description.txt  
  inflating: kaggle/sample_submission.csv  
  inflating: kaggle/sample_submission.csv.gz  
  inflating: kaggle/test.csv         
  inflating: kaggle/test.csv.gz      
  inflating: kaggle/train.csv        
  inflating: kaggle/train.csv.gz     


<a name='2'></a>
## 2 - Lựa chọn và chuẩn hóa dữ liệu cần cho việc training

In [4]:
# Đọc dữ liệu từ file train.csv
train_data = pd.read_csv("kaggle/train.csv")
# Hiển thị 5 dòng đầu tiên của dữ liệu
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# xem thucộ tính của các trường dữ liệu
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
# dữ liệu có 1460 dòng và 81 trường dữ liệu
train_data.shape

(1460, 81)

In [7]:
# lựa chọn các trường dữ liệu
train_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [8]:
features = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
X= train_data[features]
y=train_data['SalePrice']

In [9]:
X.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,TotRmsAbvGrd,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,856,854,0,1710,...,8,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,1262,0,0,1262,...,6,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,920,866,0,1786,...,6,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,961,756,0,1717,...,7,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,1145,1053,0,2198,...,9,192,84,0,0,0,0,0,12,2008


In [10]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [11]:
# kiểm tra dữ liệu rỗng
X.isnull().sum().sort_values(ascending = False)

MSSubClass       0
LotArea          0
MoSold           0
MiscVal          0
PoolArea         0
ScreenPorch      0
3SsnPorch        0
EnclosedPorch    0
OpenPorchSF      0
WoodDeckSF       0
TotRmsAbvGrd     0
KitchenAbvGr     0
BedroomAbvGr     0
HalfBath         0
FullBath         0
GrLivArea        0
LowQualFinSF     0
2ndFlrSF         0
1stFlrSF         0
YearRemodAdd     0
YearBuilt        0
OverallCond      0
OverallQual      0
YrSold           0
dtype: int64

<a name='3'></a>
## 3 - Thực hiện trainning dựa trên model của Tensorflow Keras

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu đầu vào
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
import tensorflow.keras.backend as K

# tạo hàm r_squared để đánh giá model
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Khỏi tạo model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])

# Huấn luyện model trên tập huấn luyện
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50
37/37 [==============================] - 3s 30ms/step - loss: 38884978688.0000 - mean_squared_error: 38884978688.0000 - root_mean_squared_error: 197192.7500 - mean_absolute_error: 181439.9531 - r_squared: -6.9270 - val_loss: 39652618240.0000 - val_mean_squared_error: 39652618240.0000 - val_root_mean_squared_error: 199129.6562 - val_mean_absolute_error: 178836.0781 - val_r_squared: -5.7363
Epoch 2/50
37/37 [==============================] - 0s 12ms/step - loss: 38882705408.0000 - mean_squared_error: 38882705408.0000 - root_mean_squared_error: 197186.9844 - mean_absolute_error: 181434.1719 - r_squared: -6.6962 - val_loss: 39649120256.0000 - val_mean_squared_error: 39649120256.0000 - val_root_mean_squared_error: 199120.8750 - val_mean_absolute_error: 178827.4531 - val_r_squared: -5.7358
Epoch 3/50
37/37 [==============================] - 0s 11ms/step - loss: 38876958720.0000 - mean_squared_error: 38876958720.0000 - root_mean_squared_error: 197172.4062 - mean_absolute_error: 181

In [14]:
# chạy thử mô hình với tập kiểm tra
model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", model.metrics[1].result().numpy())
print("Root Mean Squared Error:", model.metrics[2].result().numpy())
print("Mean Absolute Error:", model.metrics[3].result().numpy())
print("R-squared:", model.metrics[4].result().numpy())

10/10 [==============================] - 0s 3ms/step - loss: 10546019328.0000 - mean_squared_error: 10546019328.0000 - root_mean_squared_error: 102693.8125 - mean_absolute_error: 92084.7344 - r_squared: -0.9230
Mean Squared Error: 10546019000.0
Root Mean Squared Error: 102693.81
Mean Absolute Error: 92084.734
R-squared: -0.92295706


### Kết luận

sau khi thực hiện train model và ta được kết quả:
- Mean Squared Error: 10546019000.0
- Root Mean Squared Error: 102693.81
- Mean Absolute Error: 92084.734
- R-squared: -0.92295706

<a name='4'></a>
## 4 - Sử dụng Hyperparameter Tuning để cải tiến model

In [15]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00


In [16]:
import tensorflow as tf
from kerastuner.tuners import RandomSearch
# tạo hàm build_model
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',
                                                             values=[1e-2, 1e-3, 1e-4])),
                  loss='mse',
                  metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])
    return model

# khỏi tạo tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='kaggle',
    project_name='hoc_ml_mse11')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# Print best hyperparameters
print(tuner.get_best_hyperparameters()[0].values)

Trial 5 Complete [00h 00m 15s]
val_loss: 39626626389.333336

Best val_loss So Far: 22037066410.666668
Total elapsed time: 00h 01m 36s
{'units': 512, 'learning_rate': 0.01}


In [17]:
# lấy model tốt nhất
best_model = tuner.get_best_models(num_models=1)[0]

# chạy với tập train
best_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


Epoch 1/100
37/37 [==============================] - 2s 19ms/step - loss: 19769946112.0000 - mean_squared_error: 19769946112.0000 - root_mean_squared_error: 140605.6406 - mean_absolute_error: 128923.6719 - r_squared: -2.9777 - val_loss: 19155331072.0000 - val_mean_squared_error: 19155331072.0000 - val_root_mean_squared_error: 138402.7812 - val_mean_absolute_error: 122693.7109 - val_r_squared: -2.3036
Epoch 2/100
37/37 [==============================] - 0s 6ms/step - loss: 17287137280.0000 - mean_squared_error: 17287137280.0000 - root_mean_squared_error: 131480.5625 - mean_absolute_error: 120091.0703 - r_squared: -2.7282 - val_loss: 16743535616.0000 - val_mean_squared_error: 16743535616.0000 - val_root_mean_squared_error: 129396.8125 - val_mean_absolute_error: 114082.7109 - val_r_squared: -1.8965
Epoch 3/100
37/37 [==============================] - 0s 6ms/step - loss: 15070177280.0000 - mean_squared_error: 15070177280.0000 - root_mean_squared_error: 122760.6484 - mean_absolute_error: 11

In [18]:
# chạy thử mô hình với tập kiểm tra
best_model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model.metrics[3].result().numpy())
print("R-squared:", best_model.metrics[4].result().numpy())

10/10 [==============================] - 0s 3ms/step - loss: 1345812864.0000 - mean_squared_error: 1345812864.0000 - root_mean_squared_error: 36685.3203 - mean_absolute_error: 24583.2617 - r_squared: 0.8155
Mean Squared Error: 1345812900.0
Root Mean Squared Error: 36685.32
Mean Absolute Error: 24583.262
R-squared: 0.8155395


### Kết luận

sau khi sử dụng Hyperparameter Turning và ta được kết quả:
- Mean Squared Error: 1345812900.0
- Root Mean Squared Error: 36685.32
- Mean Absolute Error: 24583.262
- R-squared: 0.8155395


<a name='5'></a>
## 5 - Cải tiến Hyperparameter Tuning
- thay đổi giá trị max_trials=10

In [19]:
# Define tuner
tuner_2 = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='kaggle',
    project_name='hoc_ml_mse11_turn2')

# Fit tuner to data
tuner_2.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# Print best hyperparameters
print(tuner_2.get_best_hyperparameters()[0].values)

Trial 10 Complete [00h 00m 14s]
val_loss: 22899636906.666668

Best val_loss So Far: 22899636906.666668
Total elapsed time: 00h 02m 47s
{'units': 480, 'learning_rate': 0.01}


In [20]:
# lấy model tốt nhất
best_model_2 = tuner_2.get_best_models(num_models=1)[0]

# chạy với tập train
best_model_2.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
37/37 [==============================] - 2s 20ms/step - loss: 20532047872.0000 - mean_squared_error: 20532047872.0000 - root_mean_squared_error: 143290.0781 - mean_absolute_error: 131746.0938 - r_squared: -3.3669 - val_loss: 19962515456.0000 - val_mean_squared_error: 19962515456.0000 - val_root_mean_squared_error: 141288.7656 - val_mean_absolute_error: 125726.2578 - val_r_squared: -2.4475
Epoch 2/100
37/37 [==============================] - 0s 6ms/step - loss: 18067666944.0000 - mean_squared_error: 18067666944.0000 - root_mean_squared_error: 134416.0156 - mean_absolute_error: 123254.0156 - r_squared: -2.5613 - val_loss: 17539674112.0000 - val_mean_squared_error: 17539674112.0000 - val_root_mean_squared_error: 132437.4375 - val_mean_absolute_error: 117340.7812 - val_r_squared: -2.0404
Epoch 3/100
37/37 [==============================] - 0s 5ms/step - loss: 15836474368.0000 - mean_squared_error: 15836474368.0000 - root_mean_squared_error: 125843.0547 - mean_absolute_error: 11

In [21]:
# chạy thử mô hình với tập kiểm tra
best_model_2.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model_2.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model_2.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model_2.metrics[3].result().numpy())
print("R-squared:", best_model_2.metrics[4].result().numpy())

10/10 [==============================] - 0s 4ms/step - loss: 1370459008.0000 - mean_squared_error: 1370459008.0000 - root_mean_squared_error: 37019.7109 - mean_absolute_error: 24677.4453 - r_squared: 0.8098
Mean Squared Error: 1370459000.0
Root Mean Squared Error: 37019.71
Mean Absolute Error: 24677.445
R-squared: 0.80979043


### Kết luận

Sau khi sử dụng thay đổi giá trị max_trials và ta được kết quả:

- Mean Squared Error: 1370459000.0
- Root Mean Squared Error: 37019.71
- Mean Absolute Error: 24677.445
- R-squared: 0.80979043




<a name='6'></a>
## 6 - Cải tiến bằng cách dùng Hyperband

In [22]:
from kerastuner import Hyperband
# Initialize the Hyperband tuner
tuner_3 = Hyperband(build_model,
                     objective='val_mean_squared_error',
                     max_epochs=5,
                     factor=3,
                     directory='kaggle',
                     project_name='hoc_ml_mse11_turn3')

# Perform the hyperparameter search
tuner_3.search(X_train, y_train, epochs=10, validation_split=0.2)

# Get the best model
best_model_3 = tuner_3.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 00m 04s]
val_mean_squared_error: 37842165760.0

Best val_mean_squared_error So Far: 35024478208.0
Total elapsed time: 00h 00m 31s


In [23]:
# chạy thử mô hình với tập kiểm tra
best_model_3.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", best_model_3.metrics[1].result().numpy())
print("Root Mean Squared Error:", best_model_3.metrics[2].result().numpy())
print("Mean Absolute Error:", best_model_3.metrics[3].result().numpy())
print("R-squared:", best_model_3.metrics[4].result().numpy())

10/10 [==============================] - 0s 4ms/step - loss: 36704792576.0000 - mean_squared_error: 36704792576.0000 - root_mean_squared_error: 191584.9531 - mean_absolute_error: 171676.3594 - r_squared: -5.2391
Mean Squared Error: 36704793000.0
Root Mean Squared Error: 191584.95
Mean Absolute Error: 171676.36
R-squared: -5.2391396


### Kết luận

Sau khi sử dụng Hyperband để cải tiến mô hình:

- Mean Squared Error: 36704793000.0
- Root Mean Squared Error: 191584.95
- Mean Absolute Error: 171676.36
- R-squared: -5.2391396